In [ ]:
import pandas as pd
import numpy as np
import re
import os

In [ ]:
def base_no_horizontal(base):
    base = base[
        (base['porteria'].isna()) &
        (base['conjuntocerrado']==0) &
        (base['tipoinmueble']=='Casa') &
        (base['ascensor'].isna()) &
        ((base['gimmasio'].isna()) | (base['gimnasio']==0)) &
        (base['piscina'].isna()) & 
        ((base['area']<1500) & (base['area']>35)) & 
        (base['piso']<5) &
        ((base['habitaciones']<15) & (base['habitaciones']>0))
        #Si encontramos gimnasios en la geocodificación comparar con los puntos que se quitaron
    ]
    return base

In [ ]:
carpeta_path = r'C:\Users\William\Desktop\el algoritmo es correcto'
archivo_train = r'train_data.csv'
path_train = os.path.join(carpeta_path, archivo_train)

In [ ]:
path_train

In [ ]:
train = pd.read_csv(
    path_train, 
    dtype={
        "tiempodeconstruido": str,
        "vista": str,
        "tipoinmueble": str,
        "tiponegocio": str,
        'valorventa': 'int64',
        # 'area': 'int64'
    }
)

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
for columna in train.columns:
    if columna not in ['id', 'valoradministracion', 'valorventa', 'latitud', 'longitud']:
        print("{}: ".format(columna), train[columna].unique())

In [ ]:
prueba = train[
    (train['porteria'].isna()) &
    (train['conjuntocerrado']==0) &
    (train['tipoinmueble']=='Casa') &
    (train['ascensor'].isna()) &
    ((train['gimnasio'].isna()) | (train['gimnasio']==0)) &
    (train['piscina'].isna()) &
    ((train['area']<1500) & (train['area']>35)) &
    ((train['piso']<5) | (train['piso'].isna())) &
    ((train['habitaciones']<15) & (train['habitaciones']>0))
]#.groupby(['estrato','banos'], as_index=False).agg(**{'Cantidad': pd.NamedAgg('area', 'count')}).head(30)

In [ ]:
prueba.shape

In [ ]:
#Columnas a categóricas
columnas_categoricas = ['piso', 'tiempodeconstruido', 'vigilancia', 'vista', 'zonalavanderia']
for columna in columnas_categoricas:
    prueba[columna] = np.where(
        prueba[columna].notna(), prueba[columna], 'No define'
    ).astype(str)


In [ ]:
# columnas que no serán usadas
columnas_no_usar = ['ascensor', 'tipoinmueble', 'porteria', 'conjuntocerrado', 'gimnasio', 'piscina', 'valoradministracion','tiponegocio']
prueba = prueba.drop(columns = columnas_no_usar)

In [ ]:
prueba['banos'] = np.where(
    prueba['banos']==0,
    1,
    np.where(
        prueba['banos']>5,
        5,
        np.where(
            prueba['banos'].isna(),
            1,
            prueba['banos']
        )
    )
)

In [ ]:
prueba['garajes'] = np.where(
    prueba['garajes']>4,
    5,
    np.where(
        prueba['garajes']<=4,
        prueba['garajes'],
        'No define'
    )
).astype(str)

In [ ]:
prueba['deposito'] = np.where(
    prueba['deposito']==0,
    'No',
    np.where(
        prueba['deposito']==1,
        'Si',
        'No define'
    )
)

In [ ]:
prueba['piso'].unique()

In [ ]:
prueba['vista'] = np.where(
    prueba['vista'].notna(),
    prueba['vista'],
    'No defina'
)

In [ ]:
prueba['zonalavanderia'] = np.where(
    prueba['deposito'].isna(),
    'No define',
    1
).astype(str)

In [ ]:
columnas_valor0 = [
    'balcon', 'banoservicio', 'cuartoservicio', 'estudio', 'halldealcobas', 
    'remodelado', 'parqueaderovisitantes', 'saloncomunal','terraza']
prueba[columnas_valor0] = prueba[columnas_valor0].fillna(0)

In [ ]:
#PENDIENTE GARAJE
# Piso y tiempo de construido, vigilancia, vista, zonalavanderia a categorica con no define
# No sirve, piscina, gimnasio, tiponegocio
prueba.info()

In [ ]:
prueba[['habitaciones', 'banos', 'estrato']].plot(kind = 'box')

In [ ]:
prueba['y'] = prueba['valorventa']/prueba['area']

In [ ]:
#Quitar super estigmas de valores muy altos por metro cuadrado
prueba = prueba[prueba['y']<prueba['y'].quantile(0.99).astype('int64')]

In [ ]:
prueba[["y","estrato", "habitaciones"]].boxplot(by=["estrato", "habitaciones"], figsize=(10,10))

In [ ]:
import geopandas

In [ ]:
localidades_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\localidades.json'
localidades = geopandas.read_file(localidades_path, crs={'init':'epsg:4686'})

In [ ]:
# localidades_geo = localidades.plot(figsize=(10, 10), column='LocCodigo', zorder=1)

In [ ]:
estratificacion_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\manzana_estratificacion.json'
estratificacion = geopandas.read_file(estratificacion_path, crs={'init':'epsg:4686'})

In [ ]:
estratificacion = estratificacion[['ESTRATO', 'geometry']]

In [ ]:
casas = geopandas.GeoDataFrame(
    prueba, geometry=geopandas.points_from_xy(prueba.longitud, prueba.latitud), crs={'init': 'epsg:4326'})

In [ ]:
# localidades_geo = localidades.plot(color='white', edgecolor='black', figsize=(10, 10), zorder=1)
# estratificacion_geo = estratificacion.plot(ax=localidades_geo, figsize=(10, 10), zorder=2, cmap='OrRd', legend=True)
# casas_geo = casas.plot(ax=localidades_geo, figsize=(10, 10), markersize=.1, zorder=3, marker='x')

In [ ]:
estrato_intersectado = geopandas.sjoin(left_df=casas, right_df=estratificacion, how='left', op='intersects')

In [ ]:
estrato_intersectado['ESTRATO'] = np.where(
    estrato_intersectado['ESTRATO'].isna() | estrato_intersectado['ESTRATO']==0,
    np.NaN,
    estrato_intersectado['ESTRATO']
)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
train = estrato_intersectado.loc[estrato_intersectado['ESTRATO'].notna(),['ESTRATO', 'latitud', 'longitud']]

In [ ]:
X = train[['latitud', 'longitud']].values
y = train[['ESTRATO']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Se crea el modelo con la base de entrenamiento
resultados_train = {}
resultados_test = {}
for i in range(1,12):
    neigh = KNeighborsClassifier(n_neighbors=i, metric = 'euclidean')
    neigh.fit(X_train, y_train.ravel())
    resultados_train[i] = neigh.score(X_train, y_train)
    resultados_test[i] = neigh.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(list(resultados_train.keys()), list(resultados_train.values()), label='train')
ax.plot(list(resultados_test.keys()), list(resultados_test.values()), label='test')
ax.legend()
plt.show()

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3, metric = 'euclidean')
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [ ]:
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))
print("Accuracy: {}".format(accuracy_score(y_test, y_pred)))

In [ ]:
valores = estrato_intersectado.loc[estrato_intersectado['ESTRATO'].isnull(),['latitud', 'longitud']].values

In [ ]:
estrato_intersectado.loc[estrato_intersectado['ESTRATO'].isnull(),'ESTRATO'] = neigh.predict(valores)

In [ ]:
estrato_intersectado.groupby(['ESTRATO'], as_index=False, dropna=False).agg(**{'Cantidad': pd.NamedAgg('latitud','count')})

In [ ]:
a = estrato_intersectado['estrato'] - estrato_intersectado['ESTRATO']
pd.DataFrame(a, columns=['Dif Estrato']).reset_index(drop=False).rename(columns={'index':'Cantidad'}).groupby('Dif Estrato', as_index=False, dropna=False).count()

In [ ]:
estrato_intersectado.columns

In [ ]:
avaluo_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\avaluo'
avaluo = geopandas.read_file(avaluo_path, crs={'init':'epsg:4686'})

In [ ]:
avaluo.columns

In [ ]:
avaluo.shape

In [ ]:
avaluo[['AVALUO_CAT', 'AVALUO_COM']].boxplot(figsize=(10,10))

In [ ]:
pd.concat(
    [avaluo[['AVALUO_CAT', 'AVALUO_COM']].quantile(np.linspace(0,1,21)).astype('int64'),
    estrato_intersectado[['y']].quantile(np.linspace(0,1,21)).astype('int64')],
    axis=1)

In [ ]:
avaluo = avaluo[
    (avaluo['AVALUO_CAT']<avaluo['AVALUO_CAT'].quantile(0.99)) &
    (avaluo['AVALUO_COM']<avaluo['AVALUO_COM'].quantile(0.99))
]

In [ ]:
avaluo_intersectado = geopandas.sjoin(left_df=estrato_intersectado.drop(columns='index_right'), right_df=avaluo, how='left', op='intersects')

In [ ]:
avaluo_intersectado[avaluo_intersectado['AVALUO_COM'].notna()][['y', 'AVALUO_CAT', 'AVALUO_COM']].astype('int64')

In [ ]:
train = avaluo_intersectado.loc[avaluo_intersectado['AVALUO_COM'].notna()]

In [ ]:
train.columns

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import GridSearchCV

train = avaluo_intersectado.loc[avaluo_intersectado['AVALUO_COM'].notna()]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_COM']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# Se crea el modelo con la base de entrenamiento
# parameters = {
#   'n_neighbors':np.array([4,5,6]),
#   'weights':['distance'],
#   'algorithm': ['brute'],
#   'leaf_size': [1],
#   'p': np.array([2]),
#   'metric':['chebyshev']
#               }
# #Cambiar a GridSearch 
# knn = KNeighborsRegressor()
# cvknn = GridSearchCV(
#   knn, 
#   parameters, 
#   # scoring=make_scorer(mean_absolute_percentage_error), 
#   cv = 2, 
#   n_jobs=-1
# )

In [ ]:
cvknn.fit(X_train, y_train)

In [ ]:
cvknn.best_params_

In [ ]:
from sklearn.metrics import mean_absolute_error
neigh = KNeighborsRegressor(
    algorithm='brute',
    n_neighbors=5, 
    leaf_size=1,
    p=2,
    weights='distance',
    metric='chebyshev',
    n_jobs=-1)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = avaluo_intersectado.loc[avaluo_intersectado['AVALUO_COM'].isnull(),['latitud', 'longitud']].values

In [ ]:
avaluo_intersectado.loc[avaluo_intersectado['AVALUO_COM'].isnull(),'AVALUO_COM'] = neigh.predict(valores)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import GridSearchCV

train = avaluo_intersectado.loc[avaluo_intersectado['AVALUO_CAT'].notna(),['AVALUO_CAT', 'latitud', 'longitud']]
X = train[['latitud', 'longitud']].values
y = train[['AVALUO_CAT']].values
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# Se crea el modelo con la base de entrenamiento

In [ ]:
from sklearn.metrics import mean_absolute_error
neigh = KNeighborsRegressor(
    algorithm='brute',
    n_neighbors=5, 
    leaf_size=1,
    p=2,
    weights='distance',
    metric='chebyshev',
    n_jobs=-1)
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)
print("R^2: {}".format(neigh.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

In [ ]:
valores = avaluo_intersectado.loc[avaluo_intersectado['AVALUO_CAT'].isnull(),['latitud', 'longitud']].values

In [ ]:
avaluo_intersectado.loc[avaluo_intersectado['AVALUO_CAT'].isnull(),'AVALUO_CAT'] = neigh.predict(valores)

In [ ]:
avaluo_intersectado.columns

In [ ]:
columns_fill=['CP_TERR_AR', 'GRUPOP_TER']
avaluo_intersectado[columns_fill] = avaluo_intersectado[columns_fill].fillna('No define')

In [ ]:
valorref_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\valor_ref'
valorref = geopandas.read_file(valorref_path, crs={'init':'epsg:4686'})

In [ ]:
valorref = valorref[valorref['V_REF']<valorref['V_REF'].quantile(0.99)]

In [ ]:
valorref.sample(10000).plot(column='V_REF', figsize=(10,10), legend=True)

In [ ]:
valorref_intersectado = geopandas.sjoin(left_df=avaluo_intersectado.drop(columns='index_right'), right_df=valorref, how='left', op='intersects')

In [ ]:
valorref_intersectado[valorref_intersectado['id']==1204371][['id', 'V_REF', 'ANO']]#.drop_duplicates('id', keep='last')

In [ ]:
avaluo_intersectado.columns

In [ ]:
sin_columnas = [
    'id', 'estrato', 'geometry', 'index_right', 'latitud', 'longitud', 'OBJECTID', 'MANZANA_ID',
    'OBSERVACIO', 'GLOBALID', 'SHAPE_Leng', 'SHAPE_Area']
datos = avaluo_intersectado.drop(columns=sin_columnas)

In [ ]:
datos.columns

In [ ]:
estrato_intersectado2.loc[estrato_intersectado2["habitaciones"]<5,["y","ESTRATO", "habitaciones"]].boxplot(by=["ESTRATO", "habitaciones"], figsize=(10,10))

In [ ]:
avaluo_intersectado.columns

In [ ]:
avaluo_intersectado.info()

In [ ]:
columnas_eliminar_modelo = [
    'estrato', 'index_right', 'latitud', 'longitud', 
    'geometry', 'OBJECTID', 'MANZANA_ID', 'OBSERVACIO', 
    'GLOBALID', 'SHAPE_Leng', 'SHAPE_Area']
data1 = pd.get_dummies(avaluo_intersectado.drop(columns=columnas_eliminar_modelo), drop_first=True)

In [ ]:
data1.info()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

X = data1.drop(columns=['y']).values
y = data1[['y']].values

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
mod1 = DecisionTreeRegressor(
    criterion='friedman_mse', 
    splitter='best',
    max_depth=70, #70
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    max_leaf_nodes=None,
    max_features='auto', #None
    min_impurity_decrease=0
).fit(X_train, y_train)

In [ ]:
mean_absolute_percentage_error(y_train, mod1.predict(X_train))

In [ ]:
mean_absolute_percentage_error(y_test, mod1.predict(X_test))*100

In [ ]:
avaluocatastral_path = r'C:\Users\William\Desktop\el algoritmo es correcto\Datos abiertos bogota\avaluo'
avaluocatastral = geopandas.read_file(avaluocatastral_path, crs={'init':'epsg:4686'})

In [ ]:
avaluocatastral

In [ ]:
estrato_intersectado = geopandas.sjoin(left_df=casas, right_df=estratificacion, how='left', op='intersects')

In [ ]:
estrato_intersectado = estrato_intersectado.drop(columns=['index_right', 'CODIGO_MAN', 'CODIGO_ZON', 'CODIGO_CRI', 'NORMATIVA',
       'ACTO_ADMIN', 'NUMERO_ACT', 'FECHA_ACTO', 'ESCALA_CAP', 'FECHA_CAPT',
       'RESPONSABL', 'SHAPE_Leng', 'SHAPE_Area'])

In [ ]:
avaluo_intersectado = geopandas.sjoin(left_df=estrato_intersectado, right_df=avaluocatastral, how='left', op='intersects')

In [ ]:
avaluo_intersectado[
    (avaluo_intersectado['piso'].isin([1,2,3,4]))].groupby(['CP_TERR_AR', 'piso', 'GRUPOP_TER'], as_index=False, dropna=False).ESTRATO.count().ESTRATO.sum()

In [ ]:
# prueba.groupby(['habitaciones'], as_index=False, dropna=False).agg(**{'Cantidad': pd.NamedAgg('estrato', 'count')})
prueba.info()